In [1]:
import random
import tqdm
import logging
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, RepeatedKFold, KFold, StratifiedKFold

/home/wuxinchao/.conda/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def encode_seqCDR(seqCDR):
    encoding_list = []
    for i in range(len(seqCDR)):
        if seqCDR[i] == "*":
            encoding_list.append(np.zeros(5).reshape(1,5))
        else:
            encoding_list.append(af.loc[seqCDR[i]].values.reshape(1,5))
    return np.array(encoding_list).reshape(1,-1)

af = pd.read_csv("~/data/project/pMHC-TCR/library/Atchley_factors.csv")
af.index = af["Amino acid"]
af.drop(columns=["Amino acid"], inplace=True)

In [ ]:
# for dataset 230220.csv
class TCRDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        df_ng = df.copy()
        df_ng = df_ng[df_ng["HLA"] != "-"]
        df_ng["Class"] = "negative"
        df_ng["AseqCDR_3"] = df_ng["AseqCDR_3"].apply(
            lambda x: random.choice(list(set(df["AseqCDR_3"]) - set(x))))
        df_ng["BseqCDR_3"] = df_ng["BseqCDR_3"].apply(
            lambda x: random.choice(list(set(df["BseqCDR_3"]) - set(x))))
        df_pos = df[df["Class"] == "positive"]
        df = pd.concat([df_pos, df_ng], axis=0)
        df = df["HLA", "Neo", "AseqCDR_3", "BseqCDR_3", "Class"]
        seq_list = ["AseqCDR_3", "BseqCDR_3"]
        len_map = df[seq_list].applymap(len).max()
        X_feature = np.zeros((len(df), 0))
        for column in seq_list:
            df[column] = df[column].str.ljust(len_map[column], "*")
            encode_seq_result = list()
            for i in df[column]:
                encode_seq_result.append(encode_seqCDR(i))
            col_name = column + "_encode"
            df[col_name] = encode_seq_result
            col_feature = np.zeros((0, len_map[column]*5))
            for i in range(len(df)):
                col_feature = np.vstack((col_feature, df.loc[i, col_name].reshape(1, -1)))
            X_feature = np.hstack((X_feature, col_feature))

In [3]:
# for more complicated dataset which the HLA has more than 14 types
class HLAAutoEncoder_twoLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(HLAAutoEncoder_twoLayer, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim*4),
            nn.ReLU(True),
            nn.Linear(hidden_dim*4, hidden_dim*2),
            nn.ReLU(True),
            nn.Linear(hidden_dim*2, hidden_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim*2),
            nn.ReLU(True),
            nn.Linear(hidden_dim*2, hidden_dim*4),
            nn.ReLU(True),
            nn.Linear(hidden_dim*4, input_dim),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

In [ ]:
model = HLAAutoEncoder_twoLayer(input_dim=5*len(df["aaSeqHLA"].unique().max()), hidden_dim=10)
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-8)

In [ ]:
epochs = 20
output = []
losses = []
for epoch in range(epochs):
    for idx, (data) in enumerate(train_loader):
        data = Variable(data).float()
        optimizer.zero_grad()

# 230221 dataset analysis

In [4]:
df = pd.read_excel("/DATA/User/wuxinchao/project/data/seqData/pMHC-TCR_20230221_Info.xlsx")
# select the HLA:HLA-A*02:01, HLA-A*11:01
df = df[(df["HLA"] == "HLA-A*02:01") | (df["HLA"] == "HLA-A*11:01")]
# set the index of cellname and chain
df = df.set_index(['cellname', "chain"])
# extract the NeoAA, HLA, and aaSeqCDR columns
df = df[["NeoAA", "HLA", "aaSeqCDR1", "aaSeqCDR2", "aaSeqCDR3", "Class"]]
df["aaSeqCDR"] = df[df.columns[2:-1]].apply(
    # lambda x: x[0] + 'X' * (7 - len(x[0])) + x[1] + x[2],
    lambda x: '_'.join(x.dropna().astype(str)),
    axis=1
)

idx = pd.IndexSlice

df_a = df.loc[idx[:,"TRA"],]
df_a["AseqCDR"] = df_a["aaSeqCDR"]
df_a.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
# drop the chain index
df_a.index = df_a.index.droplevel(1)
# print(df_a)
df_b = df.loc[idx[:,"TRB"],]
df_b["BseqCDR"] = df_b["aaSeqCDR"]
df_b.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
# drop the chain index
df_b.index = df_b.index.droplevel(1)
# print(df_b)

# merge the TRA and TRB dataframes by cellname, HLAs, and NeoAA
df_ab = pd.merge(df_a, df_b, on=["cellname", "HLA", "NeoAA", "Class"])

df = df_ab
# select the NeoAA first 3 aa and last 3 aa as new column
df["Neo_first3"] = df["NeoAA"].str[:3]
df["Neo_last3"] = df["NeoAA"].str[-3:]
df = df.drop(columns=["NeoAA"])
# df["NeoAA"].value_counts()

/tmp/ipykernel_451163/2328245337.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a["AseqCDR"] = df_a["aaSeqCDR"]
/tmp/ipykernel_451163/2328245337.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
/tmp/ipykernel_451163/2328245337.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

SSCMGGMNQR     494
VVGAVGVGK      373
VVVGAGDVGK     143
ATAPSLSGK      118
VVVGADGVGK      99
SVCAGILSY       79
VLLSHLSYL       77
ATATAPSLSGK     76
TTAPPLSGK       64
VVGAGDVGK       42
SLMEQIPHL       18
LVTDDLLTL       14
SLLMWITQC        9
ELAGIGILTV       8
FLSEQLSIKL       6
GVLEVSHSI        6
AAGIGILTV        2
GILGFVFTL        2
NLVPMVATV        2
GTSGSPIVNR       1
LLFGYAVYV        1
ELAGIGALTV       1
ELAAIGILTV       1
LLFGYPVAV        1
YLEPGPVTV        1
SLYNTIATL        1
EAAGIGILTV       1
SLFNTIAVL        1
GILEFVFTL        1
SLLMWITQV        1
ALWGFFPVL        1
LLFGYPVYV        1
SLYNTVATL        1
Name: NeoAA, dtype: int64

In [5]:
# Determine whether to use Neo column, and the decision is not to use
# Instead, we use encoding of NeoAA
df["Neo"] = df["NeoAA"].str.slice(0,3) + "_" + df["NeoAA"].str.slice(-4,-1)
df.drop(columns=["NeoAA"], inplace=True)
df["Neo"].value_counts()

SSC_MNQ    494
VVG_GVG    373
ATA_LSG    194
VVV_DVG    143
VVV_GVG     99
SVC_ILS     79
VLL_LSY     77
TTA_LSG     64
VVG_DVG     42
SLM_IPH     18
LVT_LLT     14
SLL_ITQ     10
ELA_ILT      9
GVL_SHS      6
FLS_SIK      6
GIL_VFT      3
AAG_ILT      2
NLV_VAT      2
YLE_PVT      1
LLF_PVY      1
SLF_IAV      1
SLY_VAT      1
SLY_IAT      1
GTS_IVN      1
LLF_PVA      1
ELA_ALT      1
LLF_AVY      1
ALW_FPV      1
EAA_ILT      1
Name: Neo, dtype: int64

In [6]:
df["HLA"].value_counts()

HLA-A*11:01    1489
HLA-A*02:01     157
Name: HLA, dtype: int64

In [11]:
df["Class"].value_counts()

positive    1109
negative     537
Name: Class, dtype: int64

In [10]:
# This is the final output dataframe that we need to use
# df = df.drop(columns=["Neo"])
df.to_csv("~/data/project/data/seqData/230221.csv")

In [49]:
class TCRDataset(Dataset):
    def __init__(self, file_path):
        # super(TCRDataset, self).__init__()
        df = pd.read_csv(file_path, index_col=0)
        # get the CDR3 region
        for chain in ["AseqCDR", "BseqCDR"]:
            # df[chain+"_1"] = df[chain].str.split("_").str[0]
            # df[chain+"_2"] = df[chain].str.split("_").str[1]
            df[chain+"_3"] = df[chain].str.split("_").str[2]
            df.drop(columns=[chain], inplace=True)
        df_ps = df[df["Class"] == "positive"]
        df_ng_ex = df[df["Class"] == "negative"]
        df_ng_em = df.copy()
        df_ng_em = df_ng_em[df_ng_em["Class"] == "positive"]
        df_ng_em["AseqCDR_3"] = df_ng_em["AseqCDR_3"].apply(lambda x: random.choice(list(set(df_ng_em["AseqCDR_3"]) - set(x))))
        df_ng_em["BseqCDR_3"] = df_ng_em["BseqCDR_3"].apply(lambda x: random.choice(list(set(df_ng_em["BseqCDR_3"]) - set(x))))
        df_ng = pd.merge(df_ng_ex, df_ng_em)
        df = pd.merge(df_ps, df_ng)
        # encode the Neo_first3, Neo_last3
        for seq in ["Neo_first3", "Neo_last3"]:
            df[seq] = df[seq].apply(lambda x: encode_seqCDR(x))
        # encode the CDR3 region
        len_map = {
            "AseqCDR_3": df["AseqCDR_3"].apply(lambda x: len(x)).max(),
            "BseqCDR_3": df["BseqCDR_3"].apply(lambda x: len(x)).max(),
        }
        for chain in ["AseqCDR_3", "BseqCDR_3"]:
            length = len_map[chain]
            df[chain] = df[chain].apply(lambda x: x + "*" * (length - len(x)))
            df[chain] = df[chain].apply(lambda x: encode_seqCDR(x))
        
        # encode HLA type through one-hot encoding
        X_HLA = df["HLA"].values.reshape(-1, 1)
        HLAencoder = OneHotEncoder()
        X_HLA_encoded = HLAencoder.fit_transform(X_HLA).toarray()

        # put the features together including encoded HLA type, Neo_first3, Neo_last3, and CDR3 region
        X = np.concatenate((X_HLA_encoded, df[["Neo_first3", "Neo_last3", "AseqCDR_3", "BseqCDR_3"]].values), axis=1)
        # encode the class label
        y = df["Class"].apply(lambda x: 1 if x == "positive" else 0).values
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).float()

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [46]:
# only for test
file_path = "~/data/project/data/seqData/230221.csv"
df = pd.read_csv(file_path, index_col=0)
# get the CDR3 region
for chain in ["AseqCDR", "BseqCDR"]:
    # df[chain+"_1"] = df[chain].str.split("_").str[0]
    # df[chain+"_2"] = df[chain].str.split("_").str[1]
    df[chain+"_3"] = df[chain].str.split("_").str[2]
    df.drop(columns=[chain], inplace=True)
# delete the index
df_ps = df[df["Class"] == "positive"]
df_ng_ex = df[df["Class"] == "negative"]
df_ng_em = df.copy()
df_ng_em = df_ng_em[df_ng_em["Class"] == "positive"]
df_ng_em["AseqCDR_3"] = df_ng_em["AseqCDR_3"].apply(lambda x: random.choice(list(set(df_ng_em["AseqCDR_3"]) - set(x))))
df_ng_em["BseqCDR_3"] = df_ng_em["BseqCDR_3"].apply(lambda x: random.choice(list(set(df_ng_em["BseqCDR_3"]) - set(x))))
df_ng = pd.concat([df_ng_em, df_ng_ex], axis=0)
df_ng.index = range(len(df_ng))
df = pd.concat([df_ps, df_ng], axis=0)
# encode the Neo_first3, Neo_last3
for seq in ["Neo_first3", "Neo_last3"]:
    df[seq] = df[seq].apply(lambda x: encode_seqCDR(x))
# encode the CDR3 region
len_map = {
    "AseqCDR_3": df["AseqCDR_3"].apply(lambda x: len(x)).max(),
    "BseqCDR_3": df["BseqCDR_3"].apply(lambda x: len(x)).max(),
}
for chain in ["AseqCDR_3", "BseqCDR_3"]:
    length = len_map[chain]
    # print(length)
    df[chain] = df[chain].apply(lambda x: x + "*" * (length - len(x)))
    df[chain] = df[chain].apply(lambda x: encode_seqCDR(x).reshape(1, -1))

In [48]:
# df.loc[0, "AseqCDR_3"]
df["HLA"]

,HLA,Class,Neo_first3,Neo_last3,AseqCDR_3,BseqCDR_3
V350085868_L01_502,HLA-A*02:01,positive,"[[-1.337, -0.279, -0.544, 1.242, -1.262, -1.01...","[[-0.228, 1.399, -4.76, 0.67, -2.647, 0.26, 0....","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591..."
V350085868_L01_503,HLA-A*02:01,positive,"[[-1.337, -0.279, -0.544, 1.242, -1.262, -1.01...","[[-0.228, 1.399, -4.76, 0.67, -2.647, 0.26, 0....","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591..."
V350085868_L01_504,HLA-A*02:01,positive,"[[-1.337, -0.279, -0.544, 1.242, -1.262, -1.01...","[[-0.228, 1.399, -4.76, 0.67, -2.647, 0.26, 0....","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.228..."
V350085868_L01_505,HLA-A*02:01,positive,"[[-1.337, -0.279, -0.544, 1.242, -1.262, -1.01...","[[-0.228, 1.399, -4.76, 0.67, -2.647, 0.26, 0....","[[-1.343, 0.465, -0.862, -1.02, -0.255, -1.019...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591..."
V350085868_L01_506,HLA-A*02:01,positive,"[[-1.337, -0.279, -0.544, 1.242, -1.262, -1.01...","[[-0.228, 1.399, -4.76, 0.67, -2.647, 0.26, 0....","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591..."
...,...,...,...,...,...,...
1641,HLA-A*11:01,negative,"[[-1.337, -0.279, -0.544, 1.242, -1.262, -1.33...","[[-1.337, -0.279, -0.544, 1.242, -1.262, -0.38...","[[-1.343, 0.465, -0.862, -1.02, -0.255, 0.931,...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591..."
1642,HLA-A*11:01,negative,"[[-0.591, -1.302, -0.733, 1.57, -0.146, -0.032...","[[-0.228, 1.399, -4.76, 0.67, -2.647, -0.384, ...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.228..."
1643,HLA-A*11:01,negative,"[[-0.032, 0.326, 2.213, 0.908, 1.313, -0.032, ...","[[-0.228, 1.399, -4.76, 0.67, -2.647, -0.384, ...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.228..."
1644,HLA-A*11:01,negative,"[[-1.337, -0.279, -0.544, 1.242, -1.262, -1.33...","[[-1.337, -0.279, -0.544, 1.242, -1.262, -0.38...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.228..."


# Some useful functions and class

In [ ]:
# put the sequence are similar, (the length of the sequence are different less than 5 aa) into a batch, and then use the autoencoder to encode the HLA sequence.

class LenMatchBatchSampler(data.BatchSampler):
    def __iter__(self):
        buckets = [[] for i in range(300)]
        yielded = 0

        for idx in self.sampler:
            count_zeros = int(torch.sum(self.sampler.data_source[idx] == 0) / 5)
            buckets[count_zeros].append(idx)

            if len(buckets[count_zeros]) == self.batch_size:
                batch = list(buckets[count_zeros])
                yield batch
                buckets[count_zeros] = []

        batch = []
        leftover = [idx for bucket in buckets for idx in bucket]

        for idx in leftover:
            batch.append(idx)
            if len(batch) == self.batch_size:
                yield batch
                batch = []

        if len(batch) > 0 and not self.drop_last:
            yield batch
            

In [12]:
# not use
# random_seq_len = [random.randint(i) for i in range(5)]
test_input = torch.empty(0)
for i in range(5):
    random_seq_len = random.randint(0, 300)
    input = torch.randint(5, 100, (1, random_seq_len))
    test_input = torch.cat((test_input, input), dim=0)

In [ ]:
# # not use
# # My may need to find a proper way to encode the HLA aa sequence, because there are 10 to 20 different HLA types and some of them have variants which are just one single aa difference.
# hla_list = list(set(df["HLA"]))
# hla_list.sort()
# hla_dict = dict()
# for i in range(len(hla_list)):
#     hla_dict[hla_list[i]] = i
# # The encoding could apply autoencoder to encode the HLA sequence.
# df["HLA_encode"] = df["HLA"].map(hla_dict)
# X_feature = np.hstack((X_feature, df["HLA_encode"].values.reshape(-1,1)))

In [11]:
# torch.randint(0, 10, (3, 5))
# torch.randperm(10)
random_seq_len = random.randint(0, 300)
input = torch.randint(0, 100, (1, 5*random_seq_len))
print(input.shape)

torch.Size([1, 415])
